In [1]:
!pip install -q arxiv
!pip install -q pdfminer.six
!pip install -q PyPDF2
!pip install -q -U ragatouille
!pip install -q langchain
!pip install -q langchain-community

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.1/116.1 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━

In [3]:
import arxiv
from pdfminer.high_level import extract_text
import os
from pathlib import Path
from PyPDF2 import PdfReader
from langchain_community.document_loaders import PyPDFLoader
# change training.py - replace "from transformers import AdamW, get_linear_schedule_with_warmup" with
# from transformers import get_linear_schedule_with_warmup
# from torch.optim import AdamW
from ragatouille import RAGPretrainedModel

In [4]:
class TextRAG:
    def __init__(self):
        self.RAG = RAGPretrainedModel.from_pretrained("colbert-ir/colbertv2.0")

    def create_collection(self, paper_id, content):
        self.RAG.index(collection=[content],index_name=paper_id,max_document_length=512,split_documents=True)

    def retrieve_documents(self, query, top_k=1, threshold=0):
        results = self.RAG.search(query=query, k=top_k)
        filtered_contents = []
        for result in results:
            if result.get("score", 0) > threshold:
                filtered_contents.append(result.get("content", ""))
        return filtered_contents

In [5]:
def fetch_arxiv_paper(paper_id, download_dir = "./papers"):
    os.makedirs(download_dir, exist_ok=True)
    filename = f"{paper_id}.pdf"
    search = arxiv.Search(id_list=[paper_id])
    paper = next(arxiv.Client().results(search))
    paper.download_pdf(dirpath=download_dir, filename=filename)
    paper_path = os.path.join(download_dir, filename)
    paper_abstract = paper.summary
    paper_content = ""
    loader = PyPDFLoader(paper_path)
    pages = loader.load_and_split()
    for i in pages:
      paper_content += i.page_content
    return paper_content, paper_abstract

In [6]:
paper_id = "1605.08386v1"
content, abstract = fetch_arxiv_paper(paper_id)
print("Abstract:", abstract)
print("Content Snippet:", content[:500])

Abstract: Graphs on lattice points are studied whose edges come from a finite set of
allowed moves of arbitrary length. We show that the diameter of these graphs on
fibers of a fixed integer matrix can be bounded from above by a constant. We
then study the mixing behaviour of heat-bath random walks on these graphs. We
also state explicit conditions on the set of moves so that the heat-bath random
walk, a generalization of the Glauber dynamics, is an expander in fixed
dimension.
Content Snippet: arXiv:1605.08386v1  [math.CO]  26 May 2016
HEAT-BATH RANDOM W ALKS WITH MARKOV BASES
CAPRICE STANLEY AND TOBIAS WINDISCH
Abstract. Graphs on lattice points are studied whose edges come from a ﬁ nite set of
allowed moves of arbitrary length. We show that the diameter of these graphs on ﬁbers of a
ﬁxed integer matrix can be bounded from above by a constant. W e then study the mixing
behaviour of heat-bath random walks on these graphs. We also state explicit conditions
on the set of moves so that t

In [7]:
rag = TextRAG()
rag.create_collection(paper_id, content)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


artifact.metadata:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/405 [00:00<?, ?B/s]

...

[Apr 30, 23:41:35] [0] 		 #> Encoding 47 passages..


/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  return torch.cuda.amp.autocast() if self.activated else NullContextManager()
1it [00:00,  1.97it/s]
100%|██████████| 1/1 [00:00<00:00, 901.03it/s]

[Apr 30, 23:41:35] #> Optimizing IVF to store map from centroids to list of pids..
[Apr 30, 23:41:35] #> Building the emb2pid mapping..
[Apr 30, 23:41:35] len(emb2pid) = 15583



100%|██████████| 1024/1024 [00:00<00:00, 43107.01it/s]

[Apr 30, 23:41:36] #> Saved optimized IVF to .ragatouille/colbert/indexes/1605.08386v1/ivf.pid.pt
Done indexing!


In [8]:
query = "What is Proposition 4.11"
results = rag.retrieve_documents(query=query, top_k=3, threshold=0)
results

Loading searcher for index 1605.08386v1 for the first time... This may take a few seconds
[Apr 30, 23:41:37] #> Loading codec...
[Apr 30, 23:41:37] #> Loading IVF...
[Apr 30, 23:41:37] #> Loading doclens...


/usr/local/lib/python3.11/dist-packages/colbert/utils/amp.py:12: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler()
100%|██████████| 1/1 [00:00<00:00, 3113.81it/s]

[Apr 30, 23:41:37] #> Loading codes and residuals...



100%|██████████| 1/1 [00:00<00:00, 622.95it/s]


Searcher loaded!

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: What is Proposition 4.11, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  2054,  2003, 14848,  1018,  1012,  2340,   102,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103], device='cuda:0')
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')



['Proof. Using the isomorphism from Proposition\n4.11, we can choose a non-zero v ∈ QP such\nthat Hπ\nF,mv = v for all m ∈ M \\ M ′ and Hπ\nF,mv = 0 for all m ∈ M ′. In particular\nHπ,f\nF,Mv =\n∑\nm∈M\nf(m)Hπ\nF,mv =\n∑\nm∈M\\M′\nf(m)Hπ\nF,mv =\n∑\nm∈M\\M\nf(m)v.\nSince f is a mass function, 1 − ∑\nm∈M′ f(m) is an eigenvalue of Hπ,f\nF,M. □\nDeﬁnition 4.13. Let F ⊂ Zd and m, m′ ∈ Zd not collinear. The pair ( m, m′) has the\nintersecting ray property in F if the following holds: For any pair of rays R1, R2 parallel\nto m and any pair of rays R′\n1, R′\n2 parallel to m′ where both R1 ∩ R′\n1 and R2 ∩ R′\n2 are not\nempty, then R1 ∩ R ′\n2 ̸= ∅ implies R′\n1 ∩ R 2 ̸= ∅ and |R1| · |R ′\n1|−1 = |R2| · |R ′\n2|−1. For a\nﬁnite set M ⊂ Zd, the graph Fc(M) has the intersecting ray property if all ( m, m′) have the\nintersecting ray property in F.\nExample 4.14. The compressed ﬁber graph on [ n1] × · · · × [nd] ⊂ Zd that uses the unit\nvectors {e1, . . . , e d} as moves has the intersecting ra